In [1]:
!pip install nltk tensorflow tflearn

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/sw/arch/Debian10/EB_production/2021/software/Python/3.9.5-GCCcore-10.3.0/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import nltk

In [3]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/lcur0983/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk

In [5]:
from nltk.stem.lancaster import LancasterStemmer

In [6]:
import numpy as np

In [7]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
import tensorflow as tf

In [9]:
import json

In [10]:
import pickle

In [11]:
import random

In [13]:
#Loading intents.json
with open('intents.json') as intents:
  data = json.load(intents)

stemmer = LancasterStemmer()

# getting informations from intents.json--
words = []
labels = []
x_docs = []
y_docs = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [14]:
# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [15]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [16]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 1499  | total loss: 0.02120 | time: 0.003s
| Adam | epoch: 500 | loss: 0.02120 - acc: 1.0000 -- iter: 16/18
Training Step: 1500  | total loss: 0.02805 | time: 0.004s
| Adam | epoch: 500 | loss: 0.02805 - acc: 1.0000 -- iter: 18/18
--
INFO:tensorflow:/home/lcur0983/JHL_notebooks/CHATBOT OPTIES MERVE ÖNAL /model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

In [18]:
def chat():

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

In [ ]:
chat()



You: Hi
Bot:	Good to see you again
